- 保监会 相关性模型 2 训练 

# 基本设置

In [1]:
##load packages, needed
# encoding=utf-8

import jieba
import sys
import re
import time
import string
from sklearn import feature_extraction
from sklearn.pipeline import Pipeline,FeatureUnion

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_classif,f_classif 

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics

from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict

import joblib
%matplotlib inline
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [2]:
class StatsFeatures(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def getcnt(self,x):        
        return len(list(set(x)))
    
    def transform(self, X):
        data = []
        for x in X:
            if len(x) == 0:
                length  = 1
            else :
                length = len(x)
            data.append([len(x),self.getcnt(x),self.getcnt(x)/length])            
        return data

## 上一版模型读入

In [3]:
# 上一版模型
# from sklearn.externals import joblib
# pipeline_old = joblib.load( "model/circ_cor_0625.pkl.z") 

# 导入数据

In [4]:
folder = '20180808'

# 相关数据
corpus_cor = []
label_cor = []

filename = 'data/{0}/corpus_pre_cor_0809.txt'.format(folder)
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    corpus_cor.append(f)
    label_cor.append(1)
fid.close()
print(len(corpus_cor))
print(len(label_cor))

5776
5776


In [5]:
# 不相关数据
corpus_uncor = []
label_uncor = []

filename = 'data/{0}/corpus_pre_uncor_0809.txt'.format(folder)
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    corpus_uncor.append(f)
    label_uncor.append(0)
fid.close()
print(len(corpus_uncor))
print(len(label_uncor))

5461
5461


In [6]:
corpus_uncor[0]

'bangkok nationasia news network vendor klong thailand landmark market defended character surroundings coming criticism online post spoken defence worldfamous market foreigner critical comment news video clip titled train drive market uploaded youtube thursday april clip feature thailand famous fresh market samut songkhram province market referred folding parasol vendor fold parasol railwayside stall train move market sell kind food agricultural produce unique character location market tray remain partly moving train slowly move online comment dust train shower food foreigner complained risk accident customer tourist proximity moving train chokchai kijrak driedseafood vendor market foreigner visit market criticising vendor market clean litter urged foreigner understand local culture life boonkerd pasee sell lime market unique attracts huge tourist lamented tourist hurt trade market presence market crowded tourist fresh produce boonkerd klong train station chief pattana wongmujarin mark

# 训练模型

## 分割训练集、测试集

In [7]:
# 未加入系统中噪音
corpus = corpus_cor + corpus_uncor
label = label_cor + label_uncor
X_train, X_test, y_train, y_test = train_test_split(corpus, label, test_size=0.1, random_state=42)
print('训练集：',len(y_train))
print('训练集-各类数量：',Counter(y_train))
print('测试集：',len(y_test))
print('测试集-各类数量：',Counter(y_test))

训练集： 10113
训练集-各类数量： Counter({1: 5190, 0: 4923})
测试集： 1124
测试集-各类数量： Counter({1: 586, 0: 538})


## xgboost

### train

In [8]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('len_stats', StatsFeatures())
    ])),
    ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=2))
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.9687530900820726


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0...tate=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

### test

In [9]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.9279359430604982
             precision    recall  f1-score   support

          0       0.93      0.92      0.92       538
          1       0.93      0.93      0.93       586

avg / total       0.93      0.93      0.93      1124

confusion_matrix: 
[[497  41]
 [ 40 546]]


In [10]:
# 上一版模型 
# y_pred_class = pipeline_old.predict(X_test)
# print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
# print(metrics.classification_report(y_test, y_pred_class))
# print('confusion_matrix: ')
# print( metrics.confusion_matrix(y_test, y_pred_class))

### KFold

In [12]:
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import StandardScaler

kf = KFold(n_splits=5, shuffle=True, random_state=0)
title_content = np.array(corpus)
label = np.array(label)

In [13]:
num = 0
for train_index , test_index in kf.split(title_content):
    print('---- %s'%(num+1))
    X_train,X_test = title_content[train_index], title_content[test_index]
    y_train,y_test = label[train_index], label[test_index]
    
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('tf_idf', Pipeline([
                ('counts', CountVectorizer(max_df=0.95, min_df=2)),
                ('tf_idf', TfidfTransformer()),
                ('chi', SelectKBest(chi2, k=20000))
            ])),
            ('len_stats', StatsFeatures())
        ])),
        ('standard', StandardScaler(with_mean=False)),
        ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=8))
    ])
    pipeline.fit(X_train, y_train)
    print(pipeline.score(X_train, y_train))    
    
    y_pred_class = pipeline.predict(X_test)
    print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
    print(metrics.classification_report(y_test, y_pred_class))
    print('confusion_matrix: ')
    print( metrics.confusion_matrix(y_test, y_pred_class))    
    
    num += 1

---- 1
0.9728557125375459
accuracy_score:  0.9243772241992882
             precision    recall  f1-score   support

          0       0.92      0.93      0.93      1132
          1       0.93      0.92      0.92      1116

avg / total       0.92      0.92      0.92      2248

confusion_matrix: 
[[1051   81]
 [  89 1027]]
---- 2
0.9737456891756592
accuracy_score:  0.9150355871886121
             precision    recall  f1-score   support

          0       0.92      0.90      0.91      1073
          1       0.91      0.92      0.92      1175

avg / total       0.92      0.92      0.92      2248

confusion_matrix: 
[[ 971  102]
 [  89 1086]]
---- 3
0.9746384872080089
accuracy_score:  0.910547396528705
             precision    recall  f1-score   support

          0       0.92      0.90      0.91      1116
          1       0.91      0.92      0.91      1131

avg / total       0.91      0.91      0.91      2247

confusion_matrix: 
[[1007  109]
 [  92 1039]]
---- 4
0.9721913236929922
accura

# 模型保存

In [18]:
from sklearn.externals import joblib
joblib.dump(pipeline, "model/mfa_cor_0809.pkl.z")  

['model/mfa_cor_0809.pkl.z']

# 保存本文件

In [56]:
if 0:
    import datetime as dt
    
    def output_HTML(read_file, output_file):
        from nbconvert import HTMLExporter
        import codecs
        import nbformat
        exporter = HTMLExporter()
        # read_file is '.ipynb', output_file is '.html'
        output_notebook = nbformat.read(read_file, as_version=4)
        output, resources = exporter.from_notebook_node(output_notebook)
        codecs.open(output_file, 'w', encoding='utf-8').write(output)

    html_file_folder = 'html_files'
    if not os.path.exists(html_file_folder):
        os.makedirs(html_file_folder)

    today = dt.datetime.now().strftime('%Y%m%d')
    current_file = 'circ_cor_model_2_train.ipynb'
    output_file = 'html_files\%s_%s.html'%(os.path.splitext(current_file)[0], today)
    output_HTML(current_file, output_file)